In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from collections import defaultdict
from tqdm import tqdm  # Для прогресс-бара

# Гиперпараметры
NUM_EPISODES = 5000
ALPHA = 0.1       # Скорость обучения
GAMMA = 0.9       # Дисконтирующий фактор
EPSILON = 0.1     # Вероятность случайного действия
TARGET_VOLUME = 10  # Целевой объём жидкости в каждом сосуде

# Среда
class LiquidEnv:
    def __init__(self, num_vessels=5, target_volume=TARGET_VOLUME):
        self.num_vessels = num_vessels
        self.target_volume = target_volume
        self.state = None
    
    def reset(self):
        self.state = np.random.randint(1, 20, size=self.num_vessels)
        return tuple(self.state)
    
    def step(self, action):
        i, j, v = action
        if self.state[i] >= v:  # Переливание возможно только если достаточно жидкости
            new_state = list(self.state)
            new_state[i] -= v
            new_state[j] += v
            self.state = tuple(new_state)
        reward = -1  # Единичный штраф за действие
        done = all(v == self.target_volume for v in self.state)
        return self.state, reward, done

    def available_actions(self):
        # Возвращает все возможные действия для текущего состояния
        actions = []
        for i in range(self.num_vessels):
            for j in range(self.num_vessels):
                if i != j and self.state[i] > 0:
                    for v in range(1, self.state[i] + 1):
                        actions.append((i, j, v))
        return actions

# Q-обучение
def q_learning(env, num_episodes):
    q_table = defaultdict(float)
    rewards = []
    for episode in tqdm(range(num_episodes), desc="Обучение агента"):  # Прогресс-бар
        state = env.reset()
        total_reward = 0
        
        while True:
            # Получаем доступные действия
            actions = env.available_actions()
            
            # Выбор действия
            if np.random.rand() < EPSILON:
                action = actions[np.random.choice(len(actions))]
            else:
                q_values = [q_table[(state, a)] for a in actions]
                action = actions[np.argmax(q_values)]
            
            # Выполнение действия
            next_state, reward, done = env.step(action)
            total_reward += reward
            
            # Обновление Q-таблицы
            max_next_q = max([q_table[(next_state, a)] for a in env.available_actions()], default=0)
            q_table[(state, action)] += ALPHA * (reward + GAMMA * max_next_q - q_table[(state, action)])
            
            state = next_state
            if done:
                break
        
        rewards.append(total_reward)
    return rewards

# Обучение
env = LiquidEnv()
rewards = q_learning(env, NUM_EPISODES)

# График средней награды
window = 100
avg_rewards = [np.mean(rewards[i:i+window]) for i in range(len(rewards) - window)]

plt.plot(avg_rewards)
plt.xlabel('Эпизоды')
plt.ylabel('Средняя награда (за последние 100 эпизодов)')
plt.title('Обучение агента')
plt.show()


Обучение агента:   0%|          | 0/5000 [00:00<?, ?it/s]